## Маски байеровского шаблона для трех каналов

In [309]:
import numpy as np
from skimage import img_as_float, img_as_ubyte

In [877]:
red = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
red

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [632]:
def get_bayer_masks(n_rows, n_cols):
    n_rows_2 = int((n_rows + (n_rows % 2)) / 2 + 1)
    n_cols_2 = int((n_cols + (n_cols % 2)) / 2 + 1)
    
    red = np.array([0, 1, 0, 0], 'bool').reshape(2, 2)
    green = np.array([1, 0, 0, 1], 'bool').reshape(2, 2)
    blue = np.array([0, 0, 1, 0], 'bool').reshape(2, 2)
    
    red = np.tile(red,[n_rows_2, n_cols_2])[:n_rows, :n_cols]
    green = np.tile(green,[n_rows_2, n_cols_2])[:n_rows, :n_cols]
    blue = np.tile(blue,[n_rows_2, n_cols_2])[:n_rows, :n_cols]

    a = np.dstack((red, green, blue))
    return a

In [633]:
get_bayer_masks(6,2)

array([[[False,  True, False],
        [ True, False, False]],

       [[False, False,  True],
        [False,  True, False]],

       [[False,  True, False],
        [ True, False, False]],

       [[False, False,  True],
        [False,  True, False]],

       [[False,  True, False],
        [ True, False, False]],

       [[False, False,  True],
        [False,  True, False]]])

## Цветное изображение с неизвестными значениями

In [590]:
def get_colored_img(raw_img):
    a = get_bayer_masks(len(raw_img), len(raw_img[0]))
    a = np.dstack((a[...,0]*raw_img, a[...,1]*raw_img, a[...,2]*raw_img))
    return a

In [714]:
get_colored_img(red)[...,0]

array([[0, 2, 0],
       [0, 0, 0],
       [0, 8, 0]])

## Билинейная интерполяция

In [1210]:
def bilinear_interpolation(colored_img):

    h, w = colored_img[..., 0].shape
    res_matr = colored_img * 0

    for i in range(3):
        res = colored_img[..., i].copy()
        bayer = get_bayer_masks(h, w)[..., i]

        for j in range(1, h-1):
            for k in range(1, w-1):
                if bayer[j,k] == 0:
                    colored_img[..., i][j, k] = np.sum(res[j-1:j+2, k-1:k+2])/np.count_nonzero(bayer[j-1:j+2, k-1:k+2])
    
    res_matr[1:h-1, 1:(w-1)] = colored_img[1:h-1, 1:(w-1)]
                    
    return res_matr

In [1211]:
raw_img = np.array([[202, 150, 137, 121, 195],
                     [ 94,   0, 217,  68, 248],
                     [208, 170, 109,  67,  22],
                     [ 20,  93, 222,  54,  50],
                     [254, 252,  10, 187, 203]], dtype='uint8')
colored_img = get_colored_img(raw_img)
gt_img = np.zeros((5, 5, 3), 'uint8')
r = slice(1, -1), slice(1, -1)
gt_img[r + (0,)] = np.array([[160, 127,  94],
                              [170, 118,  67],
                              [211, 169, 127]])
gt_img[r + (1,)] = np.array([[  0,  78,  68],
                              [102, 109,  63],
                              [ 93,  66,  54]])
gt_img[r + (2,)] = np.array([[155, 217, 232],
                              [138, 219, 184],
                              [121, 222, 136]])

In [1212]:
colored_img = get_colored_img(raw_img)
img = img_as_ubyte(bilinear_interpolation(colored_img))
colored_img[...,2], img[...,2]

(array([[  0,   0,   0,   0,   0],
        [ 94, 155, 217, 232, 248],
        [  0, 138, 219, 184,   0],
        [ 20, 121, 222, 136,  50],
        [  0,   0,   0,   0,   0]], dtype=uint8),
 array([[  0,   0,   0,   0,   0],
        [  0, 155, 217, 232,   0],
        [  0, 138, 219, 184,   0],
        [  0, 121, 222, 136,   0],
        [  0,   0,   0,   0,   0]], dtype=uint8))

In [1213]:
gt_img[...,2]

array([[  0,   0,   0,   0,   0],
       [  0, 155, 217, 232,   0],
       [  0, 138, 219, 184,   0],
       [  0, 121, 222, 136,   0],
       [  0,   0,   0,   0,   0]], dtype=uint8)

In [1193]:
def compute_psnr(img_pred, img_gt):
    mask, h, w = img_pred.shape
    mse = np.sum((img_pred.astype(int) - img_gt.astype(int))**2) / (mask * h * w)
    if(mse == 0):
        raise ValueError
    else:
        return 10 * np.log10(np.max(img_gt)**2 / mse)

In [1194]:
img_pred = np.array([[[100, 117,  56],
                       [118, 156, 106]],
                      [[ 93,  13, 201],
                       [206,  15,  29]]], dtype='uint8')
img_gt = img_pred.copy()
compute_psnr(img_pred, img_gt)

ValueError: 

In [1165]:
for i in range(len(img_pred)):
    int(img_pred[i]) - int(img_gt[i])

TypeError: only size-1 arrays can be converted to Python scalars

In [1170]:
np.sum((img_pred.astype(int) - img_gt.astype(int))**2)

101